In [ ]:
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings("ignore")

In [ ]:
model = 'resnet'


time_list = [0.05, 0.06, 0.07, 0.08, 0.09, 0.1, 0.11, 0.12, 0.13, 0.14, 0.15]
arr_rate_list = [30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90]

if model == 'bert':
    time_list = [1, 1.2, 1.4, 1.6, 1.8, 2, 2.2, 2.4]
    arr_rate_list = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]

In [ ]:
data = pd.read_csv(f'{model}_infer_data_final.csv')

In [ ]:
data_all = pd.read_csv(f'{model}_infer_data_final.csv')

In [ ]:
core_vals=[4, 8, 12] #3 possible values
gpu_vals=[114750000, 318750000, 522750000, 726750000, 930750000, 1134750000, 1300500000]
cpu_vals=[422400, 729600, 1036800, 1344000, 1651200, 1958400, 2201600] #in kHz, 7 possible values
mem_vals = [665600000, 2133000000, 3199000000]
bs_vals = [1]

In [ ]:
def return_initial_samples(data, power_budget):
    mid_cores = int(np.median(core_vals))
    mid_gpu = int(np.median(gpu_vals))
    mid_cpu = int(np.median(cpu_vals))
    mid_mem = int(np.median(mem_vals))
    mid_bs = int(np.median(bs_vals))

    index_list = []

    # Extarct the sample with mid values
    mid_sample = data[(data['cores'] == mid_cores) & (data['gpu'] == mid_gpu) & (data['cpu'] == mid_cpu) & (data['mem'] == mid_mem) & (data['bs'] == mid_bs)]
    # Extract the index of the sample
    mid_index = mid_sample.index[0]
    index_list.append(mid_index)
    mid_power = mid_sample['observed_power'].values[0]

    if mid_power > power_budget:
        new_cores = int(min(core_vals))
        new_gpu = int(min(gpu_vals))
        new_cpu = int(min(cpu_vals))
        new_mem = int(min(mem_vals))
        new_bs = int(min(bs_vals))

        sample1 = data[(data['cores'] == new_cores) & (data['gpu'] == mid_gpu) & (data['cpu'] == mid_cpu) & (data['mem'] == mid_mem) & (data['bs'] == mid_bs)]
        sample2 = data[(data['cores'] == mid_cores) & (data['gpu'] == new_gpu) & (data['cpu'] == mid_cpu) & (data['mem'] == mid_mem) & (data['bs'] == mid_bs)]
        sample3 = data[(data['cores'] == mid_cores) & (data['gpu'] == mid_gpu) & (data['cpu'] == new_cpu) & (data['mem'] == mid_mem) & (data['bs'] == mid_bs)]
        sample4 = data[(data['cores'] == mid_cores) & (data['gpu'] == mid_gpu) & (data['cpu'] == mid_cpu) & (data['mem'] == new_mem) & (data['bs'] == mid_bs)]
        sample5 = data[(data['cores'] == mid_cores) & (data['gpu'] == mid_gpu) & (data['cpu'] == mid_cpu) & (data['mem'] == mid_mem) & (data['bs'] == new_bs)]

        index_list.append(sample1.index[0])
        index_list.append(sample2.index[0])
        index_list.append(sample3.index[0])
        index_list.append(sample4.index[0])
        index_list.append(sample5.index[0])

        pair1 = index_list[0:2]
        pair2 = index_list[0:1] + index_list[2:3]
        pair3 = index_list[0:1] + index_list[3:4]
        pair4 = index_list[0:1] + index_list[4:5]
        pair5 = index_list[0:1] + index_list[5:6]

        pairs = [pair1, pair2, pair3, pair4, pair5]
    
    else:
        new_cores = int(max(core_vals))
        new_gpu = int(max(gpu_vals))
        new_cpu = int(max(cpu_vals))
        new_mem = int(max(mem_vals))
        new_bs = int(max(bs_vals))

        sample1 = data[(data['cores'] == new_cores) & (data['gpu'] == mid_gpu) & (data['cpu'] == mid_cpu) & (data['mem'] == mid_mem) & (data['bs'] == mid_bs)]
        sample2 = data[(data['cores'] == mid_cores) & (data['gpu'] == new_gpu) & (data['cpu'] == mid_cpu) & (data['mem'] == mid_mem) & (data['bs'] == mid_bs)]
        sample3 = data[(data['cores'] == mid_cores) & (data['gpu'] == mid_gpu) & (data['cpu'] == new_cpu) & (data['mem'] == mid_mem) & (data['bs'] == mid_bs)]
        sample4 = data[(data['cores'] == mid_cores) & (data['gpu'] == mid_gpu) & (data['cpu'] == mid_cpu) & (data['mem'] == new_mem) & (data['bs'] == mid_bs)]
        sample5 = data[(data['cores'] == mid_cores) & (data['gpu'] == mid_gpu) & (data['cpu'] == mid_cpu) & (data['mem'] == mid_mem) & (data['bs'] == new_bs)]

        index_list.append(sample1.index[0])
        index_list.append(sample2.index[0])
        index_list.append(sample3.index[0])
        index_list.append(sample4.index[0])
        index_list.append(sample5.index[0])

        pair1 = index_list[0:2]
        pair2 = index_list[0:1] + index_list[2:3]
        pair3 = index_list[0:1] + index_list[3:4]
        pair4 = index_list[0:1] + index_list[4:5]
        pair5 = index_list[0:1] + index_list[5:6]

        pairs = [pair1, pair2, pair3, pair4, pair5]

    return pairs

In [ ]:
def fit_lines(data, pairs, zero_list):
    flag = False
    mode = ['cores', 'gpu','cpu','mem', 'bs']
    slopes = []
    for i in range(len(pairs)):
        # Fit a time and power line for each pair
        sample1 = data.iloc[pairs[i][0]]
        sample2 = data.iloc[pairs[i][1]]
        
        sample1['cores'] = sample1['cores']*100
        sample1['cpu'] = sample1['cpu']/1000
        sample1['mem'] = sample1['mem']/1000000
        sample1['gpu'] = sample1['gpu']/1000000

        sample2['cores'] = sample2['cores']*100
        sample2['cpu'] = sample2['cpu']/1000
        sample2['mem'] = sample2['mem']/1000000
        sample2['gpu'] = sample2['gpu']/1000000
        # print(sample1)
        # print(sample2)

        x1 = sample1[mode[i]]
        x2 = sample2[mode[i]]

        yt1 = sample1['observed_time']
        yt2 = sample2['observed_time']

        yp1 = sample1['observed_power']
        yp2 = sample2['observed_power']

        m_time = (yt2 - yt1)/(x2 - x1)
        c_time = yt1 - m_time*x1

        m_power = (yp2 - yp1)/(x2 - x1)
        c_power = yp1 - m_power*x1

        slope = m_time/m_power

        if abs(yp2 - yp1) < 0.2:
            slope = 0
            print("Power under 1W for mode {dim}, setting slope to 0".format(dim=mode[i]))
        # print("Mode : ", mode[i])
        # m_time_formated = "{:.3f}".format(m_time)
        # c_time_formated = "{:.3f}".format(c_time)
        # m_power_formated = "{:.3f}".format(m_power)
        # c_power_formated = "{:.3f}".format(c_power)
        # print(f"Time Line: y = {m_time_formated}x + {c_time_formated}")
        # print(f"Power Line: y = {m_power_formated}x + {c_power_formated}")
        # print("B/A =","{:.3f}".format( m_time/m_power))
        slopes.append(slope)
        # print("\n")
    

    slopes = [abs(x) for x in slopes]
    # change any value greater than 100 to 0 in slopes list

    # for i in range(len(slopes)):
    #     if slopes[i] > 100:
    #         slopes[i] = 0
    #         print("Slope of {dim} is too high, setting to 0".format(dim=mode[i]))
    #         #append the pairs with high slope to a list
    #         high_slope_list.append(pairs[i])
        
    # find the index of elements in zero_list
    zero_index = [mode.index(x) for x in zero_list]
    # print("Setting slope of {dim} to 0".format(dim=zero_list))
    # set the slopes of index in zero_index to 0
    for i in zero_index:
        slopes[i] = 0

    # round the slopes to 2 decimal places
    slopes_print = [round(x, 2) for x in slopes]
    print("Slopes (Cores, GPU, CPU, Memory, Bs): ", slopes_print)
    if sum(slopes) == 0:
        flag = True
    # return the index of the minimum value in the slopes list
    max_index = slopes.index(max(slopes))
    max_dim = mode[max_index]
    
    return max_index, max_dim, flag


In [ ]:
def bin_search(core_vals, cpu_vals, gpu_vals, mem_vals, bs_vals, dim, step, cores, cpu, gpu, mem, bs):
    # Using binary search to find the optimal value for the dimension

    
    if dim == 'cores':
        vals = core_vals
        value = cores
    elif dim == 'cpu':
        vals = cpu_vals
        value = cpu
    elif dim == 'gpu':
        vals = gpu_vals
        value = gpu
    elif dim == 'mem':
        vals = mem_vals
        value = mem
    else:
        vals = bs_vals
        value = bs

    if step=='next':    
        # print("Mode: Next")
        if len(vals) == 2:
            value = vals[1]
            vals = vals[1:]

        elif len(vals) == 1:
            value = vals[0]
            vals = vals[0:]

        else:    
            dim_index = vals.index(value)
            # print("Dim Index: ", dim_index)
            last_index = len(vals)-1
            # print("Last Index: ", last_index)
            new_index = int(np.ceil((dim_index + last_index)/2))
            value = vals[new_index]
            vals = vals[dim_index+1:]


    else:
        # print("Mode: Prev")
        if len(vals) == 1 or len(vals) == 2:
            value = vals[0]
            vals = vals[0:]

        else:
            dim_index = vals.index(value)
            new_index = int(np.floor(dim_index/2))
            value = vals[new_index]
            vals = vals[:dim_index]

    return value, vals


In [ ]:
def main(data, power_budget, max_count):

    index_list = []
    zero_list = []
    # high_slope_list = []
    visited_dim = []
    core_vals=[4, 8, 12] #3 possible values
    gpu_vals=[114750000, 318750000, 522750000, 726750000, 930750000, 1134750000, 1300500000]
    cpu_vals=[422400, 729600, 1036800, 1344000, 1651200, 1958400, 2201600] #in kHz, 7 possible values
    mem_vals = [665600000, 2133000000, 3199000000]
    if model == 'bert':
        bs_vals = [1, 4, 8, 16, 32]
    else:
        bs_vals = [1, 4, 16, 32, 64]

    pairs = return_initial_samples(data, power_budget)
    for i in pairs:
        index_list.append(i[0])
        index_list.append(i[1])
    print("Initial Pairs: ", pairs)
    mid_index = index_list[0]
    # print("Initial Index: ", mid_index)
    mid_power = data.iloc[mid_index]['observed_power']
    print("Initial Power: ", mid_power)
    print("\n")
    count = 1
    # while True:
    while count < max_count:
        
        max_index, max_dim, flag = fit_lines(data, pairs, zero_list)
        print("Dimension with max slope: ", max_dim)
        visited_dim.append(max_dim)
        # print("Max Dim: ", max_dim)
        if flag:
            break

        next_pair = pairs[max_index]

        if count == 1:
            curr_sample = data.iloc[next_pair[0]]
            curr_cores = curr_sample['cores']
            curr_cpu = curr_sample['cpu']
            curr_gpu = curr_sample['gpu']
            curr_mem = curr_sample['mem']
            curr_bs = curr_sample['bs']
            curr_power = curr_sample['observed_power']

        # else:
        #     curr_sample = data.iloc[next_pair[1]]
        #     curr_cores = curr_sample['cores']
        #     curr_cpu = curr_sample['cpu']
        #     curr_gpu = curr_sample['gpu']
        #     curr_mem = curr_sample['mem']
        #     curr_power = curr_sample['observed_power']

        # print("Current Cores, CPU, GPU, Memory: ",curr_cores, curr_cpu, curr_gpu, curr_mem)
        # print("\n")
        # print("Current Power: ", curr_power)

        if curr_power < power_budget:
            value, vals = bin_search(core_vals, cpu_vals, gpu_vals, mem_vals, bs_vals, max_dim, 'next', curr_cores, curr_cpu, curr_gpu, curr_mem, curr_bs)
            print("New Dim value: ", value)
            print("Updated dim list: ", vals)
            if len(vals) == 1:
                zero_list.append(max_dim)
                print("Setting slope of {dim} to 0".format(dim=max_dim))

            if max_dim == 'cores':
                curr_cores = value
                core_vals = vals
            elif max_dim == 'cpu':
                curr_cpu = value
                cpu_vals = vals
            elif max_dim == 'gpu':
                curr_gpu = value
                gpu_vals = vals
            elif max_dim == 'mem':
                curr_mem = value
                mem_vals = vals
            else:
                curr_bs = value
                bs_vals = vals


        else:
            value, vals = bin_search(core_vals, cpu_vals, gpu_vals, mem_vals, bs_vals, max_dim, 'prev', curr_cores, curr_cpu, curr_gpu, curr_mem, curr_bs)
            print("New Dim value: ", value)
            print("Updated dim list: ", vals)
            if len(vals) == 1:
                zero_list.append(max_dim)
                print("Setting slope of {dim} to 0".format(dim=max_dim))

            if max_dim == 'cores':
                curr_cores = value
                core_vals = vals
            elif max_dim == 'cpu':
                curr_cpu = value
                cpu_vals = vals
            elif max_dim == 'gpu':
                curr_gpu = value
                gpu_vals = vals
            elif max_dim == 'mem':
                curr_mem = value
                mem_vals = vals
            else:
                curr_bs = value
                bs_vals = vals

        # retrive new index from data
        new_sample = data[(data['cores'] == int(curr_cores)) & (data['gpu'] == int(curr_gpu)) & (data['cpu'] == int(curr_cpu)) & (data['mem'] == int(curr_mem)) & (data['bs'] == int(curr_bs))]
        curr_power = new_sample['observed_power'].values[0]
        new_index = new_sample.index[0]
        index_list.append(new_index)
        # print("New Index at "+str(count)+" iteration: ",new_index)


        if curr_power < power_budget:
            pairs[max_index] = [next_pair[0], new_index]
        else:
            pairs[max_index] = [new_index, next_pair[1]]

        print("Pairs at "+str(count)+" iteration: ",pairs) 
        # divide the curr_cpu by 1000, curr_mem by 1000000 and curr_gpu by 1000000
        curr_cpu_print = curr_cpu/1000
        curr_gpu_print = curr_gpu/1000000
        curr_mem_print = curr_mem/1000000
        print("Current Cores, CPU, GPU, Memory, Bs at "+str(count)+" iteration: ",curr_cores, curr_cpu_print, curr_gpu_print, curr_mem_print, curr_bs)
        print("Power at "+str(count)+" iteration: ",curr_power)

        print("\n")
        count+=1

    visited_dim = list(set(visited_dim))

    return index_list, len(set(index_list)), visited_dim
    

In [ ]:
# data_all[data_all['bs']==64].sort_values(by='observed_time', ascending=True)

In [ ]:
core_vals=[4, 8, 12] #3 possible values
gpu_vals=[114750000, 318750000, 522750000, 726750000, 930750000, 1134750000, 1300500000]
cpu_vals=[422400, 729600, 1036800, 1344000, 1651200, 1958400, 2201600] #in kHz, 7 possible values
mem_vals = [665600000, 2133000000, 3199000000]
bs_vals = [1]

In [ ]:
data = data[data['bs'] == 1]
data.reset_index(drop=True, inplace=True)

In [ ]:
index_list, _, _ = main(data, 34, 15)

In [ ]:
def next_bs_picker(index_list, power_budget, time_budget, arr_rate, data_all):
    data = data_all.iloc[index_list]
    # print("Data: ", data)
    all_data = data_all.copy()
    all_data['time_scaled'] = all_data['observed_time']/1000
    all_data['time_cond1'] = (all_data['bs'].astype(int))/arr_rate + all_data['time_scaled']
    all_data['time_cond2'] = all_data['time_scaled']*arr_rate
    data['time_scaled'] = data['observed_time']/1000
    data['time_cond1'] = (data['bs'].astype(int))/arr_rate + data['time_scaled']
    data['time_cond2'] = data['time_scaled']*arr_rate

    # filter the data based on the power budget and time_cond1
    data = data[(data['observed_power'] <= power_budget) & (data['time_cond1'] <= time_budget)]
    data.sort_values(by='observed_time', inplace=True)
    
    i = 1
    for row in data.iterrows():

        cores, cpu, gpu, mem = row[1][['cores', 'cpu', 'gpu', 'mem']]
        # print cores, cpu, gpu, mem
        print("Cores, CPU, GPU, Memory: ", cores, cpu, gpu, mem)
        bs4 = data_all[(data_all['cores'] == int(cores)) & (data_all['cpu'] == int(cpu)) & (data_all['gpu'] == int(gpu)) & (data_all['mem'] == int(mem)) & (data_all['bs'] == 4)]
        bs4['time_scaled'] = bs4['observed_time']/1000
        bs4['time_cond1'] = (bs4['bs'].astype(int))/arr_rate + bs4['time_scaled']
        bs4['time_cond2'] = bs4['time_scaled']*arr_rate
        bs4_index = bs4.index[0]
        print("Cond1 value: ", all_data.iloc[bs4_index]['time_cond1'])
        print("Cond2 value: ", all_data.iloc[bs4_index]['time_cond2'])
        cond1 = all_data.iloc[bs4_index]['time_cond1'] <= time_budget
        print("Condition 1 for iteration ", i, " : ", cond1)
        cond2 = all_data.iloc[bs4_index]['time_cond2'] <= 4
        print("Condition 2 for iteration ", i, " : ", cond2)
        
        if cond1 and cond2 and all_data.iloc[bs4_index]['observed_power'] <= power_budget:
            print("Found solution at iteration: ", i)
            return bs4_index, i
        i+=1
        
    print("No solution found in BS4 as well")
    return None, 0

In [ ]:
# data.iloc[index_list]

In [ ]:
power_budget = 40
time_budget = 0.1
arr_rate = 60
index_list, _, _ = main(data, power_budget, 15)
bs4_index = next_bs_picker(index_list, power_budget, time_budget, arr_rate, data_all)

In [ ]:
# data_all.iloc[bs4_index]['observed_time'], data_all.iloc[bs4_index]['observed_power']

In [ ]:
# data = data.iloc[index_list]
# data.drop_duplicates(inplace=True)

In [ ]:
# data['time_scaled'] = data['observed_time']/1000
# data['time_cond1'] = (data['bs'].astype(int))/arr_rate + data['time_scaled']
# data['time_cond2'] = data['time_scaled']*arr_rate

In [ ]:
# data.sort_values(by='observed_time', inplace=True)

In [ ]:
# # bs 4 for the given cores, cpu, gpu, mem of data
# bs4_data = pd.DataFrame(columns=data.columns)
# for row in data.iterrows():
#     cores, cpu, gpu, mem = row[1][['cores', 'cpu', 'gpu', 'mem']]
#     bs4 = data_all[(data_all['cores'] == int(cores)) & (data_all['cpu'] == int(cpu)) & (data_all['gpu'] == int(gpu)) & (data_all['mem'] == int(mem)) & (data_all['bs'] == 4)]
#     bs4['time_scaled'] = bs4['observed_time']/1000
#     bs4['time_cond1'] = (bs4['bs'].astype(int))/arr_rate + bs4['time_scaled']
#     bs4['time_cond2'] = bs4['time_scaled']*arr_rate
#     bs4_data = pd.concat([bs4_data, bs4])


In [ ]:
if model == 'bert':
    power_list = np.arange(10,61,1)
else:
    power_list = np.arange(10,51,1)

In [ ]:
data_all[data_all['bs']==1].sort_values(by='observed_time', ascending=True)

In [ ]:
def pick_best_powermode(index_list, data, power_budget, time_budget, a, num_pwds):
    # filter data based on the index_list
    filtered_data = data.iloc[index_list]
    # drop duplicates
    filtered_data = filtered_data.drop_duplicates()
    all_data = filtered_data.copy()
    # filter data based on the power budget
    filtered_data = filtered_data[filtered_data['observed_power'] <= power_budget]
    count1 = len(filtered_data)
    filtered_data['time_scaled'] = filtered_data['observed_time']/1000.0
    filtered_data['time_cond1'] = (filtered_data['bs'].astype(int))/a + filtered_data['time_scaled']
    filtered_data['time_cond2'] = filtered_data['time_scaled']*a

    filtered_data = filtered_data[(filtered_data['time_cond1'] <= time_budget)]
    count2 = len(filtered_data)
    filtered_data = filtered_data[(filtered_data['time_cond2'] <= filtered_data['bs'])]
    count3 = len(filtered_data)
    # sort the data based on the observed time
    filtered_data = filtered_data.sort_values(by='time_scaled')
    # pick the first sample
    if filtered_data.empty:
        print("No valid power mode found in BS1")
        bs4_index, tries = next_bs_picker(index_list, power_budget, time_budget, a, data_all)
        if bs4_index == None:
            print("No valid power mode found in BS4")
            # print(all_data)
            temp_data = all_data[all_data['observed_power'] > power_budget]
            if temp_data.empty:
                all_data = all_data.sort_values(by='observed_time', ascending=True)
                best_sample = all_data.iloc[0]
                return best_sample, num_pwds + tries, 0
            else:
                temp_data = temp_data.sort_values(by='observed_time', ascending=True)
                best_sample = temp_data.iloc[0]
                return best_sample, num_pwds + tries, 0
            
        best_sample = data_all.iloc[bs4_index]
        return best_sample, num_pwds + tries, 1
        
    else:
        best_sample = filtered_data.iloc[0]
        return best_sample, num_pwds, 1

In [ ]:
result_df = pd.DataFrame(columns=['cores', 'cpu', 'gpu', 'mem', 'bs', 'algo_time', 'algo_power','power_budget','num_pwds','visited_dim','tries', 'time_budget', 'arr_rate','soln'])
# high_slope_df = pd.DataFrame(columns=['high_slope_list','power_budget'])
avg_pwmds = []

for arr_rate in arr_rate_list:
    for time_budget in time_list:
        for power_budget in power_list:
            print("Power Budget: ", power_budget)
            index_list, num_pwds, visited_dim = main(data, power_budget, 15)
            # high_slope_df.loc[len(high_slope_df)] = [high_slope_list, power_budget]

            avg_pwmds.append(num_pwds)
            # print(index_list)

            best_sample, tries, soln = pick_best_powermode(index_list, data, power_budget, time_budget, arr_rate, num_pwds)
            if best_sample is None:
                result_df.loc[len(result_df)] = [0, 0, 0, 0, 0, 0, 0, power_budget, 0, 0, tries, time_budget, arr_rate, soln]
            else:
                result_df.loc[len(result_df)] = [best_sample['cores'], best_sample['cpu'], best_sample['gpu'], best_sample['mem'], best_sample['bs'], best_sample['observed_time'], best_sample['observed_power'], power_budget, num_pwds, len(visited_dim), tries, time_budget, arr_rate, soln]


        print("Average Power Modes: ", np.mean(avg_pwmds))

In [ ]:
def find_optimal_pm(df, pmax, tmax, a):
    #filter rows where interleaved_power<=pmax, sort in ascending order of interleaved_time
    df = df[df['observed_power'] <= pmax]
    df['time_scaled'] = df['observed_time']/1000.0
    df['time_cond1'] = (df['bs'].astype(int))/a + df['time_scaled']
    df['time_cond2'] = df['time_scaled']*a

    df = df[(df['time_cond1'] <= tmax)]
    df = df[(df['time_cond2'] <= df['bs'])]
    
    df = df.sort_values(by='time_scaled', ascending=True)
    #return top1 powermode, time and power
    return df.iloc[0]['cores'], df.iloc[0]['cpu'], df.iloc[0]['gpu'], df.iloc[0]['mem'], df.iloc[0]['bs'], df.iloc[0]['observed_time'], df.iloc[0]['observed_power']

In [ ]:
# data_all

In [ ]:
dis = result_df['num_pwds']

np.mean(dis)

In [ ]:
optimal_df = pd.DataFrame(columns=['power_budget', 'cores', 'cpu', 'gpu', 'mem', 'bs', 'optim_time', 'optim_power','time_budget', 'arr_rate','soln'])

for arr_rate in arr_rate_list:
    for time_budget in time_list:
        for power_budget in power_list:
            try:
                cores, cpu, gpu, mem, bs, algo_time, algo_power = find_optimal_pm(data_all, power_budget, time_budget, arr_rate)
                optimal_df.loc[len(optimal_df)] = [power_budget, cores, cpu, gpu, mem, bs, algo_time, algo_power, time_budget, arr_rate, 1]
            except:
                print("No Power Mode Found")
                optimal_df.loc[len(optimal_df)] = [power_budget, 0, 0, 0, 0, 0, 0, 0, time_budget, arr_rate, 0]

In [ ]:
# merge on power_budget and time_budget
final_df = pd.merge(result_df, optimal_df, on=['power_budget', 'time_budget', 'arr_rate'], suffixes=('_algo', '_optimal'))

In [ ]:
final_df

In [ ]:
final_df.sample(20)[['algo_time', 'optim_time', 'algo_power', 'optim_power', 'power_budget', 'time_budget', 'arr_rate', 'soln_algo', 'soln_optimal']]

In [ ]:
time = int(time_budget*1000)

In [ ]:
final_df.to_csv(f'{model}_bs_lr_exp_dim_infer_backtrack_multi_al.csv', index=False)